## **Algoritmo Para Detecção de Imagens de Faces DeepFakes Por Meio de Aprendizado de Máquina**
## Universidade Federal de São Paulo - UNIFESP

### Disciplina: Inteligência Artificial
### Professor: Fábio Faria
### Integrantes:
###  -  Marco Antonio Coral dos Santos
###  -  Raphael Damasceno Rocha de Moraes

# **1. Introdução**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sb
import torch 
from torchvision import models
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from torchvision.utils import make_grid
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
import shutil
import os
import warnings
warnings.filterwarnings("ignore")

code_dir = "/kaggle/working/code"
model_dir = "/kaggle/working/model"
output_dir = "/kaggle/working/output"

if not os.path.exists(code_dir):
    os.mkdir(code_dir)

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
shutil.copyfile(src="/kaggle/input/modelos/convnext.py", 
                dst="/kaggle/working/code/convnext.py")
shutil.copyfile(src="/kaggle/input/modelos/convnext_tiny_1k_224_ema.pth", 
                dst="/kaggle/working/model/convnext_tiny_1k_224_ema.pth")
shutil.copyfile(src="/kaggle/input/modelos/vit_b_16-c867db91.pth", 
                dst="/kaggle/working/model/vit_b_16-c867db91.pth")

os.chdir("/kaggle/working/code")


from convnext import ConvNeXt


# **2. Banco de Dados**

In [ ]:
local_arquivos='/kaggle/input/140k-real-and-fake-faces'
treino = pd.read_csv(local_arquivos + "/train.csv")
teste = pd.read_csv(local_arquivos + "/test.csv")
print(teste.shape)
print(treino.shape)

## **2.1. Arquiteturas de Rede**

In [ ]:
#Instanciando o modelo ConvNeXt 
def ConvNeXt_model():
    model_conv=ConvNeXt()
    state_dict = torch.load('/kaggle/working/model/convnext_tiny_1k_224_ema.pth')
    model_conv.load_state_dict(state_dict["model"])
    
    return model_conv

def ViT_model():
    model_vit=vit_b_16(pretrained=True)
    return model_vit


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
  criterion = nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
train_acc, teste_acc, train_loss, teste_loss = [], [], [], []
train_precision, teste_precision, train_recall, teste_recall = [], [], [], []
train_f1, teste_f1 = [], []
df = pd.DataFrame(columns=['Modelo','Experimento','Epoch', 'Train ACC', 'Train Loss', 'Train F1', 'Test ACC', 'Test Loss', 'Test F1'])

# **3. Metodologia**

## **3.1. Aumento de Dados**

### **3.1.1. Processamento das Imagens**

In [ ]:
def full_data_transform(model_type, data_fraction, batch_size):
    # Transformações de dados
    if model_type== 'convnext':
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif model_type == 'vit':
        transform = ViT_B_16_Weights.IMAGENET1K_V1.transforms()
            
            
    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    full_train_dataset = ImageFolder(local_arquivos + "/train", transform=transform)
    full_test_dataset = ImageFolder(local_arquivos + "/test", transform=transform)
            
    # Determinar o número de objetos a serem selecionados
    num_train_data = int(len(full_train_dataset) * data_fraction)
    num_test_data = int(len(full_test_dataset) * data_fraction)
    
    

    # Selecionar aleatoriamente os objetos para os conjuntos de dados
    train_indices = random.sample(range(len(full_train_dataset)), num_train_data)
    test_indices = random.sample(range(len(full_test_dataset)), num_test_data)

    # Criar os datasets com a seleção aleatória de objetos
    train_dataset = torch.utils.data.Subset(full_train_dataset, train_indices)
    test_dataset = torch.utils.data.Subset(full_test_dataset, test_indices)
    
    # Criação dos dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/2), shuffle=False)
        
        
    return train_dataloader, test_dataloader

def ft_data_transform(model_type, data_fraction, batch_size):
    # Transformações de dados
    if model_type== 'convnext':
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    
    elif model_type == 'vit':
        transform = ViT_B_16_Weights.IMAGENET1K_V1.transforms()

    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    full_test_dataset = ImageFolder(local_arquivos + "/test", transform=transform)
            
    # Determinar o número de objetos a serem selecionados
    num_test_data = int(len(full_test_dataset) * data_fraction)
    

    # Selecionar aleatoriamente os objetos para os conjuntos de dados
    test_indices = random.sample(range(len(full_test_dataset)), num_test_data)

    # Criar os datasets com a seleção aleatória de objetos
    test_dataset = torch.utils.data.Subset(full_test_dataset, test_indices)
    
    # Criação dos dataloaders
    test_dataloader = DataLoader(test_dataset, batch_size=int(batch_size/2), shuffle=False)

    return test_dataloader

### **3.1.2. AutoAugment**

In [ ]:
def AutoAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.AutoAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.AutoAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    
    return augmented_train_dataloader
    

### **3.1.3. RandAugment**

In [ ]:
def RandAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
        
    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    
    return augmented_train_dataloader
    

### **3.1.4. Auto+Rand Augment**

In [ ]:
def Auto_RandAugment_transform(model_type, train_indices, batch_size):
    if model_type== 'convnext':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.AutoAugment(),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    elif model_type == 'vit':
        augmentation_transforms = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BILINEAR),
            transforms.CenterCrop(224),
            transforms.AutoAugment(),
            transforms.RandAugment(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
        ])
        
    local_arquivos='/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake'
    augmented_train_dataset = ImageFolder(local_arquivos + "/train", transform=augmentation_transforms)
    augmented_train_dataset = torch.utils.data.Subset(augmented_train_dataset, train_indices)
    augmented_train_dataloader = DataLoader(augmented_train_dataset, batch_size=batch_size, shuffle=True)
    
    return augmented_train_dataloader

## **3.2. Treinamento e Teste**

In [ ]:
 # Função de treino genérica
def train(model, dataloader, criterion, optimizer, scheduler, device, ft, epoch, exp, model_type):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    y_true,y_pred=[], []
    
    loop = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, (images, labels) in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        if ft==False:
            for param in model.parameters():
                param.requires_grad=False
                
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.step()
            
            for param in model.parameters():
                param.requires_grad=True
        else:    
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
                
        running_loss += loss.item()
        predicted = outputs.argmax(dim = 1)
    
        y_true.extend(labels.cpu().tolist())
        y_pred.extend(predicted.cpu().tolist())

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        loop.set_description(f"[Epoch {(epoch+1)}]")
        loop.set_postfix(loss=loss.item())
        
    if scheduler:
        scheduler.step()
        
    train_loss = running_loss / len(dataloader.dataset)  
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

    print(f"Train Loss: {train_loss:.6f} | Train Accuracy: {(accuracy * 100):.2f}% | Train F1-Score: {f1:.6f}")
    
    model_name= f'model_{model_type}_params_exp_{exp}.pth'
    torch.save(model.state_dict(), os.path.join('/kaggle/working/model', model_name))
   
    return train_loss, accuracy, f1

In [ ]:
# Função de teste genérica
def test(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    y_pred, y_true= [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            

            running_loss += loss.item()
            predicted = outputs.argmax(dim = 1)
            
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())
            
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(list(dataloader.dataset))
    test_accuracy = accuracy = accuracy_score(y_true, y_pred)
    precision, recall, test_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)
    
    print(f"Test Loss: {test_loss:.6f} | Test Accuracy: {(test_accuracy * 100):.2f}% | Test F1-Score: {test_f1:.6f}")
    return test_loss, test_accuracy, test_f1


In [ ]:
# Função genérica para treinar, testar
def train_model(model_type, exp,model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, device, num_epochs, ft, num):
    model=model.to(device)
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for epoch in range(num_epochs):
        print('----------------------------------------------------------------------------')
        train_loss, train_accuracy, train_f1 = train(model, train_dataloader, criterion, optimizer, scheduler, device, ft, epoch, exp, model_type)
        test_loss, test_accuracy,test_f1 = test(model, test_dataloader, criterion, device)
        val=str(num)+str(epoch+1)
        df.loc[val]=[model_type, exp, epoch+1, train_accuracy, train_loss, train_f1, test_accuracy, test_loss, test_f1]
        df.to_csv('metricas.csv', index = False)
        print('\n')
        
        
        
    return train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1


## **3.3. Protocolo Experimental**

Aplicando protocolo experimental para os cenários resultantes das permutações dos Aumentos de Dados (AutoAugment e RandAugment) e Fine-Tuning
1. Sem Fine-Tuning
2. Com Fine-Tuning
3. Sem Fine Tuning e com AutoAugment
4. Com Fine Tuning e com AutoAugment
5. Sem Fine Tuning e com RandAugment
6. Com Fine Tuning e com RandAugment
7. Sem Fine Tuning e com AutoAugment e RandAugment
8. Com Fine Tuning e com AutoAugment e RandAugment

In [ ]:
def run_scenario(model_type, scenario, data_fraction, num_epochs=10, batch_size=32, learning_rate=0.001):
    num_classes = 2  
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    num_train_data = int((100000) * data_fraction)
    num_test_data = int((20000) * data_fraction)
    
    train_indices = random.sample(range(100000), num_train_data)
    test_indices = random.sample(range(20000), num_test_data)
    
    print(f'Número de objetos de treino: {num_train_data}')
    print(f'Número de objetos de teste: {num_test_data}')
    print("============================================================================")
    
    if scenario == 1:
        
        train_dataloader, test_dataloader= full_data_transform(model_type, data_fraction, batch_size)
        
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=1

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=9
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")
        
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 1 - ConvNeXt (Sem Fine-Tuning)")
        elif model_type == 'vit':
            print("Experimento 1 - ViT (Sem Fine-Tuning)")
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,1, model, train_dataloader, test_dataloader,
                                                                                    criterion, optimizer, scheduler, device, 1, False, num)
    elif scenario == 2:
        
        train_dataloader, test_dataloader= full_data_transform(model_type, data_fraction, batch_size)
        
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=2

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=10
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
             print("Experimento 2 - ConvNeXt (Com Fine-Tuning)")
        elif model_type == 'vit':
            print("Experimento 2 - ViT (Com Fine-Tuning)")
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,2, model, train_dataloader, test_dataloader,
                                                                                    criterion, optimizer, scheduler, device, num_epochs, True, num)
    elif scenario == 3:
        
        augmented_train_dataloader = AutoAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)
        
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=3

        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=11
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 3 - ConvNeXt (Sem Fine-Tuning e com AutoAugment)")
        elif model_type == 'vit':
            print("Experimento 3 - ViT (Sem Fine-Tuning e com AutoAugment)")
        
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,3, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=4
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=12
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 4 - ConvNeXt (Com Fine-Tuning e com AutoAugment)")
        elif model_type == 'vit':
            print("Experimento 4 - ViT (Com Fine-Tuning e com AutoAugment)")
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,4, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)
    
    elif scenario == 4:
        augmented_train_dataloader = RandAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)
        
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=5
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=13
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
  
        if model_type == 'convnext':
            print("Experimento 5 - ConvNeXt (Sem Fine-Tuning e com RandAugment)")
        elif model_type == 'vit':
            print("Experimento 5 - ViT (Sem Fine-Tuning e com RandAugment)")    
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,5, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)
        
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=6
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=14
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 6 - ConvNeXt (Com Fine-Tuning e com RandAugment)")
        elif model_type == 'vit':
            print("Experimento 6 - ViT (Com Fine-Tuning e com RandAugment)")
        
        
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,6, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num) 

    elif scenario == 5:
        augmented_train_dataloader = Auto_RandAugment_transform(model_type, train_indices, batch_size)
        test_dataloader=ft_data_transform(model_type, data_fraction, batch_size)
        
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=7
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=15
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 7 - ConvNeXt (Sem Fine-Tuning e com RandAugment e AutoAugment)")
        elif model_type == 'vit':
            print("Experimento 7 - ViT (Sem Fine-Tuning e com RandAugment e AutoAugment)")
            
        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,7, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, 1, False, num)
      
        # Selecionar o modelo
        if model_type == 'convnext':
            model = ConvNeXt_model()
            model.head = nn.Linear(model.head.in_features, num_classes)  # Substituir num_classes pelo número correto de classes
            num=8
            
        elif model_type == 'vit':
            model = ViT_model()
            model.heads=nn.Linear(768,2)
            num=16
        else:
            raise ValueError("O parâmetro 'model_type' deve ser 'convnext' ou 'vit'.")

        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

        if model_type == 'convnext':
            print("Experimento 8 - ConvNeXt (Com Fine-Tuning e com RandAugment e AutoAugment)")
        elif model_type == 'vit':
            print("Experimento 8 - ViT (Com Fine-Tuning e com RandAugment e AutoAugment)")

        train_loss, train_accuracy, train_f1, test_loss, test_accuracy,test_f1 = train_model(model_type,8, model, augmented_train_dataloader, test_dataloader,
                                                                                   criterion, optimizer, scheduler, device, num_epochs, True, num)
    else:
        raise ValueError("O parâmetro 'scenario' deve ser um número entre 1 e 5.")



## **3.4. Avaliação de Desempenho**

In [ ]:
import matplotlib.pyplot as plt

def plot_experiment_graphs(df, experiment_number):
    experiment_df = df[df['Experimento'] == experiment_number]

    # Plot epoch x train F1
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train F1'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Train F1')
    plt.title(f'Experimento {experiment_number} - Train F1')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x train accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train ACC'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Acurácia de Treino')
    plt.title(f'Experimento {experiment_number} - Acurácia de Treino')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x train loss
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Train Loss'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Perda de Treino')
    plt.title(f'Experimento {experiment_number} - Perda de Treino')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x test F1
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test F1'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Test F1')
    plt.title(f'Experimento {experiment_number} - Test F1')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x test accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test ACC'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Acurácia de Teste')
    plt.title(f'Experimento {experiment_number} - Acurácia de Teste')
    plt.tight_layout()
    plt.show()
    
    # Plot epoch x test loss
    plt.figure(figsize=(8, 6))
    plt.plot(experiment_df['Epoch'], experiment_df['Test Loss'], marker='o')
    plt.xlabel('Epoch')
    plt.ylabel('Perda de Teste')
    plt.title(f'Experimento {experiment_number} - Perda de Teste')
    plt.tight_layout()
    plt.show()

def plot_general_graphs(df):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

    # Plot epoch x train F1
    ax1 = axes[0, 0]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax1.plot(experiment_df['Epoch'], experiment_df['Train F1'], marker='o', label=f'Experimento {experiment_number}')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Train F1')
    ax1.set_title('Geral - Train F1')
    ax1.legend()

    # Plot epoch x train loss
    ax2 = axes[0, 1]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax2.plot(experiment_df['Epoch'], experiment_df['Train Loss'], marker='o', label=f'Experimento {experiment_number}')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Perda de Treino')
    ax2.set_title('Geral - Perda de Treino')
    ax2.legend()

    # Plot epoch x test F1
    ax3 = axes[1, 0]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax3.plot(experiment_df['Epoch'], experiment_df['Test F1'], marker='o', label=f'Experimento {experiment_number}')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Test F1')
    ax3.set_title('Geral - Test F1')
    ax3.legend()

    # Plot epoch x test loss
    ax4 = axes[1, 1]
    for experiment_number in df['Experimento'].unique():
        experiment_df = df[df['Experimento'] == experiment_number]
        ax4.plot(experiment_df['Epoch'], experiment_df['Test Loss'], marker='o', label=f'Experimento {experiment_number}')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('Perda de Teste')
    ax4.set_title('Geral - Perda de Teste')
    ax4.legend()

    plt.tight_layout()
    plt.show()


In [ ]:

run_scenario('convnext',1,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)


In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(df)

In [ ]:
run_scenario('convnext',2,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)


In [ ]:
run_scenario('convnext',3,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)


In [ ]:
run_scenario('convnext',4,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
run_scenario('convnext',5,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
run_scenario('vit',1,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
run_scenario('vit',2,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
run_scenario('vit',3,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
run_scenario('vit',4,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
run_scenario('vit',5,0.1, num_epochs=10, batch_size=16, learning_rate=0.0001)

In [ ]:
mean_df = df.groupby(['Modelo', 'Experimento']).mean()
print(mean_df)

In [ ]:
df.to_csv('metricas.csv', index = False)